In [ ]:
### My first data pipeline project, I made the rookie mistake of overwriting most of my steps in the same file. I've recovered what I could just to have something on my github to help the reader see into my process taht led to the dashboard


In [ ]:
### One of many attempts to clean CSV's

import csv

def reformat_csv(input_file, output_file):
    with open(input_file, 'r', newline='', encoding='utf-8') as infile, \
         open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.reader(infile, quotechar='"', escapechar='\\', doublequote=True, skipinitialspace=True)
        writer = csv.writer(outfile, quotechar='"', quoting=csv.QUOTE_MINIMAL, escapechar='\\', lineterminator='\n')
        
        current_row = []
        line_number = 0
        
        for row in reader:
            line_number += 1
            if not row:  # Skip empty rows
                continue
                
            if current_row:  # Continuing a multi-line field
                current_row.extend(row)
            else:
                current_row = row
            
            # Check if the last field is properly closed (ends with a quote)
            if current_row and current_row[-1].endswith('"'):
                writer.writerow(current_row)
                print(f"Written complete row at line {line_number}")
                current_row = []
            else:
                print(f"Buffering row at line {line_number} - incomplete quote detected")

        if current_row:  # Handle any remaining rows
            writer.writerow(current_row)
            print(f"Written final row at line {line_number}")

    print(f"Reformatting complete. Check {output_file}.")

# Usage with explicit paths
if __name__ == "__main__":
    input_file = '/Users/sam/Documents/Academics/Data/Udemy Dash/edx_courses.csv'  # Update this path
    output_file = '/Users/sam/Documents/Academics/Data/Udemy Dash/edx_courses_new.csv'  # Update this path
    reformat_csv(input_file, output_file)

In [ ]:
## Querying through bigquery 
import pandas as pd
from google.cloud import bigquery
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

# Set the environment variable for authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/sam/Documents/Academics/Data/UdemyDash/planar-effect-452317-h6-597d463b43bb.json"

# Set up your BigQuery client
client = bigquery.Client()

# Write your query (with corrected syntax)
query = """ 

SELECT title, price, course_effort, n_enrolled, institution
FROM `planar-effect-452317-h6.Udemy_Courses.udemy` 
WHERE subject == 'Computer Science'
ORDER BY price DESC LIMIT 15

"""

# Execute the query and load the result into a DataFrame
df = client.query(query).to_dataframe()

# Improve display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)

# Display the result
print(df)


In [ ]:
## saving cleaned queried tables to new csv's...
# Set the working directory to the correct folder
os.chdir("/Users/sam/Documents/Academics/Data/UdemyDash/")

# Read the CSV with flexible parsing
df = pd.read_csv("edX_courses_cleaned.csv", quotechar='"', on_bad_lines='skip', encoding='utf-8')

# Save the cleaned DataFrame to a new CSV
df.to_csv("edX_courses_cleaned_2.csv", index=False, quoting=1)  # quoting=1 ensures fields with special chars are quoted
print("Cleaned CSV saved as 'edX_courses_cleaned.csv'")


In [ ]:
## pushing tables to drive to be visualized in tableau
import pandas as pd
from google.cloud import bigquery
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

# Set the environment variable for authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/sam/Documents/Academics/Data/UdemyDash/planar-effect-452317-h6-597d463b43bb.json"

# Set up your BigQuery client
client = bigquery.Client()

# Step 2: Query the BigQuery table
query = """
SELECT *
FROM `planar-effect-452317-h6.Udemy_Courses.distinct_subjects`
"""
query_job = client.query(query)
df = query_job.to_dataframe()

# Step 3: Export the DataFrame to a CSV file
csv_file_path = "/Users/sam/Documents/Academics/Data/UdemyDash/distinct_subjects.csv"
df.to_csv(csv_file_path, index=False)
print(f"CSV file saved as: {csv_file_path}")

# Step 4: Authenticate and set up Google Drive
gauth = GoogleAuth()
# This will prompt you to authenticate via browser the first time
gauth.LocalWebserverAuth()  # Creates local webserver and auto handles authentication
drive = GoogleDrive(gauth)

# Step 5: Upload the CSV file to Google Drive
file_title = "distinct_subjects.csv"
file = drive.CreateFile({'title': file_title})
file.SetContentFile(csv_file_path)
file.Upload()
print(f"File uploaded to Google Drive with title: {file_title}")

